In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from datasketch import MinHash, MinHashLSH
import string
import re

def preprocess_text(text):
    """Preprocessa una frase rimuovendo punteggiatura e rendendo il testo in minuscolo."""
    text = text.lower()
    text = re.sub(f"[{string.punctuation}]", "", text)
    return text

def compute_minhash(sentences, num_perm=128):
    """Calcola MinHash per una lista di frasi."""
    minhashes = []
    for sentence in sentences:
        m = MinHash(num_perm=num_perm)
        for word in sentence.split():
            m.update(word.encode('utf8'))
        minhashes.append(m)
    return minhashes

def find_most_similar(dataset, sample_dataset, num_perm=128, threshold=0.8):
    """
    Trova la frase più simile per ogni frase nel campione all'interno del dataset completo usando LSH.

    Parameters:
    - dataset: DataFrame contenente `lan_code` e `sentence` (dataset completo)
    - sample_dataset: DataFrame contenente il campione di frasi
    - num_perm: Numero di permutazioni per MinHash
    - threshold: Soglia per la similarità LSH
    """
    # Preprocessing del testo
    dataset['sentence_processed'] = dataset['sentence'].apply(preprocess_text)
    sample_dataset['sentence_processed'] = sample_dataset['sentence'].apply(preprocess_text)

    # Costruzione di MinHash e LSH per l'intero dataset
    minhashes = compute_minhash(dataset['sentence_processed'], num_perm=num_perm)
    lsh = MinHashLSH(threshold=threshold, num_perm=num_perm)

    # Aggiunge MinHash al LSH per l'intero dataset
    for i, mh in enumerate(minhashes):
        lsh.insert(f"sentence_{i}", mh)

    # Ricerca della frase più simile nel dataset completo per ogni frase nel campione
    similar_sentences = []
    for i, row in sample_dataset.iterrows():
        original_sentence = row['sentence_processed']
        mh = MinHash(num_perm=num_perm)
        for word in original_sentence.split():
            mh.update(word.encode('utf8'))

        # Trova frasi simili usando LSH
        results = lsh.query(mh)

        if results:
            # Esclude la stessa frase (se presente)
            results = [r for r in results if r != f"sentence_{i}"]

            # Calcola la frase più simile tra i risultati usando cosine similarity
            candidate_indices = [int(r.split("_")[1]) for r in results]
            candidates = dataset.iloc[candidate_indices]['sentence_processed']

            vectorizer = TfidfVectorizer().fit([original_sentence] + candidates.tolist())
            vectors = vectorizer.transform([original_sentence] + candidates.tolist())
            similarities = cosine_similarity(vectors[0:1], vectors[1:]).flatten()
            best_match_index = candidate_indices[similarities.argmax()]
            similar_sentences.append((row['sentence'], dataset.iloc[best_match_index]['sentence']))
        else:
            similar_sentences.append((row['sentence'], None))

    return similar_sentences

# Esempio di utilizzo
if __name__ == "__main__":
    # Carica il dataset
    file_path = "data/filtered_language_detection.csv"  # Sostituisci con il percorso del file CSV
    dataset = pd.read_csv(file_path)

    # Preleva lo 0.1% del dataset casualmente
    sample_dataset = dataset.sample(frac=0.001, random_state=42)

    # Trova le frasi più simili solo sul campione rispetto al dataset completo
    results = find_most_similar(dataset, sample_dataset, num_perm=128, threshold=0.99)

    # Stampa i risultati
    for original, similar in results:
        print(f"Original: {original}\nMost similar: {similar}\n")


ValueError: The number of bands are too small (b < 2)